In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sncosmo

from astropy import units as u

from lightcurvelynx.astro_utils.unit_utils import flam_to_fnu, fnu_to_flam
from lightcurvelynx.astro_utils.passbands import Passband, PassbandGroup
from lightcurvelynx.models.sncomso_models import SncosmoWrapperModel

## Comparing Passband Objects

We start by confirming that the (no-preprocessed) passbands are the same whether we load them through LightCurveLynx (via sncosmo) or directly from sncosmo.  Note the (unprocessed) transmission tables have the exact same range and sampling of wavelength points and the same transmission curve (as shown by the plot).

In [ ]:
# Load the passbands from sncosmo as both sncosmo and lynx objects
sn_pb_obj = sncosmo.get_bandpass(f"lsstr")
lynx_pb_obj = Passband.from_sncosmo("LSST", "r", sn_pb_obj)
lynx_pb_group = PassbandGroup([lynx_pb_obj])

# Print out the range information for the underlying passband data
print(f"sncosmo passbands: [{sn_pb_obj.wave[0]}, {sn_pb_obj.wave[-1]}] with {len(sn_pb_obj.wave)} points")
print(
    f"lynx passbands: [{lynx_pb_obj._loaded_table[0, 0]}, {lynx_pb_obj._loaded_table[-1, 0]}] with {len(lynx_pb_obj._loaded_table)} points"
)
print(
    f"lynx processed range: [{lynx_pb_obj.processed_transmission_table[0, 0]}, {lynx_pb_obj.processed_transmission_table[-1, 0]}] with {len(lynx_pb_obj.processed_transmission_table)} points"
)


# Plot the passbands
plt.figure(figsize=(10, 6))
plt.plot(sn_pb_obj.wave, sn_pb_obj.trans, label="sncosmo")
plt.plot(lynx_pb_obj._loaded_table[:, 0], lynx_pb_obj._loaded_table[:, 1], label="lynx")
plt.xlabel("Wavelength (Angstrom)")
plt.ylabel("Transmission")
_ = plt.legend()

Of course the processed transmission table looks very different because we convert it from throughput values to system response values.

## Comparing Models and SEDs.

We can create models via either sncosmo or lightcurvelynx and get the exact same sed in fnu.

In [ ]:
times = np.arange(61420.0, 61451.0, 1)
wavelengths = sn_pb_obj.wave

# Create a supernova model
t0 = 61427.662972820726
redshift = 0.014308532704595672
x0 = 0.00048338940570825635
x1 = 0.2161431811959595
c = -0.014308532704595672
saltpars = {"x0": x0, "x1": x1, "c": c, "z": 0.0, "t0": t0}

sn_model = sncosmo.Model("salt3")
sn_model.update(saltpars)

flam_sncosmo = sn_model.flux(times, wavelengths)
fnu_sncosmo = flam_to_fnu(
    flam_sncosmo,
    wavelengths,
    wave_unit=u.AA,
    flam_unit=u.erg / u.second / u.cm**2 / u.AA,
    fnu_unit=u.nJy,
)

# Compute the SED from the LightCurveLynx model
lynx_model = SncosmoWrapperModel("salt3", **saltpars)
lynx_state = lynx_model.sample_parameters(num_samples=1)
fnu_lynx = lynx_model.evaluate_sed(times, wavelengths, lynx_state)

# Plot the SEDs from both models. The graphs should be identical.
plt.plot(wavelengths, fnu_sncosmo[0, :], label="sncosmo")
plt.plot(wavelengths, fnu_lynx[0, :], label="lightcurvelynx")
_ = plt.legend()

The SED's look identical in flam as well. 

In [ ]:
flam_lynx = fnu_to_flam(
    fnu_lynx,
    wavelengths,
    wave_unit=u.AA,
    flam_unit=u.erg / u.second / u.cm**2 / u.AA,
    fnu_unit=u.nJy,
)

# Plot the SEDs from both models. The graphs should be identical.
plt.plot(wavelengths, flam_sncosmo[0, :], label="sncosmo")
plt.plot(wavelengths, flam_lynx[0, :], label="lightcurvelynx")
plt.legend()

## Differing Bandpass Values

But if we try to get the bandpass information from both codes, they are different. The LightCurveLynx values are slightly larger than the corresponding sncosmo values.

In [ ]:
sn_bandflux_flam = sn_model.bandflux("lsstr", times)
lynx_bandflux_fnu = lynx_model.evaluate_bandfluxes(
    lynx_pb_group,
    times,
    ["r"] * len(times),
    state=lynx_state,
)
print("lynx bandflux (fnu):", lynx_bandflux_fnu)

sn_bandflux_fnu = sn_model.bandflux("lsstr", times, zp=31.4, zpsys="ab")
print("sncosmo bandflux (fnu):", sn_bandflux_fnu)

print("Ratio (lynx/sn):", lynx_bandflux_fnu / sn_bandflux_fnu)

plt.figure(figsize=(10, 6))
plt.plot(times, lynx_bandflux_fnu, label="lynx")
plt.plot(times, sn_bandflux_fnu, label="sncosmo")
plt.xlabel("Time (MJD)")
plt.ylabel("Bandflux (nJy)")
_ = plt.legend()

## The Integration

We can examine how each of the two packages performs the integrations. We break out the code from sncosmo so we can modify it. First let's reproduce the integration code and test that it gives the same answer.

In [ ]:
def sncosmo_bandflux(flux, band):
    # Set up wavelength grid. Spacing (dwave) evenly divides the bandpass,
    # closest to 5 angstroms without going over.
    wave, dwave = sncosmo.utils.integration_grid(band.minwave(), band.maxwave(), 5.0)
    trans = band(wave)

    resampled_flux = np.interp(wave, band.wave, flux)

    return np.sum(wave * trans * resampled_flux) * dwave / sncosmo.constants.HC_ERG_AA


sn_bandflux_flam = sn_model.bandflux("lsstr", times)
sn_bandflux_flam2 = [sncosmo_bandflux(flam_sncosmo[i, :], sn_pb_obj) for i in range(flam_sncosmo.shape[0])]
print("SN Cosmo FLAM Bandflux (auto):", sn_bandflux_flam)
print("SN Cosmo FLAM Bandflux (manual):", sn_bandflux_flam2)

Now that we have the function isolated, let's look at the sampled wavelengths compared to those of the processed transmission table from LightCurveLynx: 

In [ ]:
sn_processed_waves, dwave = sncosmo.utils.integration_grid(sn_pb_obj.minwave(), sn_pb_obj.maxwave(), 5.0)
lcl_processed_waves = lynx_pb_obj.processed_transmission_table[:, 0]

# Print out the range information for the underlying passband data
print(" Metric |   SN   |  Lynx  |")
print(f" Start  | {sn_processed_waves[0]:5.1f} | {lynx_pb_obj.processed_transmission_table[0, 0]:5.1f} |")
print(f" End    | {sn_processed_waves[-1]:5.1f} | {lynx_pb_obj.processed_transmission_table[-1, 0]:5.1f} |")
print(f" Points | {len(sn_processed_waves):5d}  | {len(lynx_pb_obj.processed_transmission_table):5d}  |")

Wait! Why does the processed sncosmo wavelengths have one less point?

It is because sncosmo's transmission table uses rectangular integration. Each bin is sampled in the center. We can modify the LightCurveLynx code to do the same thing.

In [ ]:
lynx_pb_obj2 = Passband.from_sncosmo("LSST", "r", sn_pb_obj, rect_interp=True)
lynx_pb_group2 = PassbandGroup([lynx_pb_obj2])

print(" Metric |   SN   |  Lynx  |")
print(f" Start  | {sn_processed_waves[0]:5.1f} | {lynx_pb_obj2.processed_transmission_table[0, 0]:5.1f} |")
print(f" End    | {sn_processed_waves[-1]:5.1f} | {lynx_pb_obj2.processed_transmission_table[-1, 0]:5.1f} |")
print(f" Points | {len(sn_processed_waves):5d}  | {len(lynx_pb_obj2.processed_transmission_table):5d}  |")

Now the sampled points and integration method match up. However the results diverge even more!

In [ ]:
lynx_bandflux_fnu2 = lynx_model.evaluate_bandfluxes(
    lynx_pb_group2,
    times,
    ["r"] * len(times),
    state=lynx_state,
)
print("sncosmo bandflux (fnu):", sn_bandflux_fnu)
print("lynx bandflux (fnu):", lynx_bandflux_fnu2)
print("Ratio (lynx/sn):", lynx_bandflux_fnu2 / sn_bandflux_fnu)

So we can be relatively confident that it is not the mechanics of the integration that is causing the difference.

## Normalization

What about the normalization? 

### In LightCurveLynx

In LightCurveLynx we compute the transmission table by first scaling all the values by the wavelenth and then normalizing the area under the curve to one. Let's call the original table values $T_i$ (for throughput) and the LightCurveLynx processed values $N_i$ (for normalized system response). Let $W_i$ be the corresponding wavelength value in Angstroms (from [here](https://github.com/lincc-frameworks/lightcurvelynx/blob/ad1c98b43c4b11533b6417ca97f666677370ffa8/src/lightcurvelynx/astro_utils/passbands.py#L1222)):

$N_i = \frac{\frac{T_i}{W_i}}{\sum_j (\delta_j * \frac{T_j}{W_j})}$

where $\delta_i$ is the width of the bin (in Angstroms)

Thus a rectangular bandpass integration with measured flux $f_i$ in each bin, would compute (from [here](https://github.com/lincc-frameworks/lightcurvelynx/blob/ad1c98b43c4b11533b6417ca97f666677370ffa8/src/lightcurvelynx/astro_utils/passbands.py#L1283)):

$bandflux_{fnu} = \sum_i  (\delta_i * f_i * N_i) = \sum_i (\delta_i * (\frac{f_i * \frac{T_i}{W_i})}{\sum_j (\delta_j * \frac{T_j}{W_j})})$

Assuming an equally spaced grid of wavelength samples ($\delta_i = \delta$ $\forall i$), we get:

$bandflux_{fnu} = \sum_i (\frac{\frac{T_i}{W_i}}{\sum_i (\frac{T_i}{W_i})})$

Let's consider a very basic case of a perfect step function filter where $T_i$=1.0 when $W_i \in [5000, 7000]$ and 0.0 otherwise that is regularly sampled a $\delta$=1.0:

$N_i = \frac{\frac{1}{W_i}}{\sum_j (\frac{1}{W_j})}$

In [ ]:
w_i = np.arange(5000, 7001, 1, dtype=float)
t_i = np.ones_like(w_i)

step_pb = Passband(
    np.column_stack((w_i, t_i)),
    survey="test",
    filter_name="r",
    delta_wave=1.0,
    trim_quantile=None,
    units="A",
)

plt.plot(step_pb.processed_transmission_table[:, 0], step_pb.processed_transmission_table[:, 1])
plt.xlabel("Wavelength (Angstrom)")
plt.ylabel("Transmission")
_ = plt.title("Step Passband")

Thus the weight we give flux values (normalized system response) in the integration *decreases* with wavelength.

Let's look at a few step function SEDs.

In [ ]:
num_times = 5
num_waves = len(w_i)

ax = plt.subplot(111)
ax.set_xlabel("Wavelength (Angstrom)")
ax.set_ylabel("FNU (nJy)")

step_fnus = np.zeros((num_times, num_waves), dtype=float)
for i in range(num_times):
    start_ind = 2 * i * (num_waves // (2 * num_times))
    end_ind = (2 * i + 1) * (num_waves // (2 * num_times))
    step_fnus[i, start_ind:end_ind] = 1.0
    ax.plot(w_i, step_fnus[i, :], label=f"t={i}")
_ = ax.legend()

In [ ]:
step_bandfluxes = step_pb.fluxes_to_bandflux(step_fnus)

plt.plot(np.arange(num_times), step_bandfluxes, marker="o")
plt.xlabel("Time")
plt.ylabel("Integrated Value")
_ = plt.title("Step Function Bandfluxes (LightCurveLynx)")

### In Sncosmo

In contrast sncosmo does not transform the throughput into system response until after the integration (from [here](https://github.com/sncosmo/sncosmo/blob/c0309761e4516530e02b6bec7128d6366050ded1/sncosmo/models.py#L119)) and it *increases* the weight of the flux values with wavelength.

$bandflux_{flam} = \frac{1}{k} \sum_i (W_i * T_i * f_i)$

where $k$ is the constant `HC_ERG_AA` = 1.9864458571489284e-08.  The bandflux is then converted from flam to fnu as (from [here](https://github.com/sncosmo/sncosmo/blob/c0309761e4516530e02b6bec7128d6366050ded1/sncosmo/models.py#L158)):

$bandflux_{fnu} = bandflux_{flam} * 10^{(0.4 * zp)} / magsys.zpbandflux$

Given the zeropoint ($zp$) and the magnitude system's zeropoint bandflux ($magsys.zpbandflux$). For LSST's red filter, we use $zp$=31.4 and $magsys.zpbandflux$=536998.199084923 for a total scaling factor of 6761252.7451452995:

$bandflux_{fnu} = \frac{1}{k'} \sum_i (W_i * T_i * f_i)$

In [ ]:
magsys = sncosmo.get_magsystem("ab")
zpnorm = 10.0 ** (0.4 * 31.4) / magsys.zpbandflux("lsstr")
print("magsys.zpbandflux: ", magsys.zpbandflux("lsstr"))
print("ZP Norm:", zpnorm)

We can create the same "step" filter as the in the LightCurveLynx section by change our `sncosmo_bandflux` function to always use a transition value of 1.0 for every bin.

In [ ]:
def sncosmo_step_bandflux(flux, waves):
    if waves.shape != flux.shape:
        raise ValueError("Wavelength and flux arrays must have the same shape")

    dwave = waves[1] - waves[0]
    if not np.allclose(np.diff(waves), dwave):
        raise ValueError("Wavelengths must be evenly spaced")
    trans = np.ones_like(waves)

    return np.sum(waves * trans * flux) * dwave / sncosmo.constants.HC_ERG_AA


step_bandfluxes2 = [sncosmo_step_bandflux(step_fnus[i, :], w_i) for i in range(num_times)]

plt.plot(np.arange(num_times), step_bandfluxes2, marker="o")
plt.xlabel("Time")
plt.ylabel("Integrated Value")
_ = plt.title("Step Function Bandfluxes (sncosmo)")